In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from object_detection.utils import dataset_util


import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json
import os

from glob import glob


ImportError: No module named cv2

In [2]:
img_dir = [y for x in os.walk('/hdd3/dhj_container/DB/data_oh/train-images') for y in glob(os.path.join(x[0], '*.png'))]

In [3]:
print(len(img_dir), img_dir[0])

1800 /hdd3/dhj_container/DB/SSD_data/img0000/101_Torn_295_196_394_293.png


In [4]:
random.shuffle(img_dir)
print(img_dir[0])
train, test = train_test_split(img_dir, test_size=0.2)
print(len(train), train[0])
print(len(test), test[0])

/hdd3/dhj_container/DB/SSD_data/img0006_re_Rotated_dataset/10032_Torn_325_323_488_480.png
1440 /hdd3/dhj_container/DB/SSD_data/img0000_re_Affine_dataset/20059_Torn_122_109_298_295.png
360 /hdd3/dhj_container/DB/SSD_data/img0000_re_Rotated_dataset/10029_Torn_136_389_254_501.png


In [5]:
def create_tf(in_dir):
    # TODO(user): Populate the following variables from your example.
    height = 512 # Image height
    width = 512 # Image width
    filename = str.encode(in_dir.split('/')[-1]) # Filename of the image. Empty if image is not from file
    image_format = b'png' # b'jpeg' or b'png'
    
    str_fname = in_dir.split('/')[-1]
    xy_str = str_fname.replace('.png','')
    xy_par = xy_str.split('_')
    x1 = int(xy_par[2])
    y1 = int(xy_par[3])
    x2 = int(xy_par[4])
    y2 = int(xy_par[5])
    
    file = open(in_dir, 'rb')
    encoded_image_data = file.read()
    
    
    xmins = [x1/width] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [x2/width] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [y1/height] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [y2/height] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [b'torn'] # List of string class name of bounding box (1 per box)
    classes = [1] # List of integer class id of bounding box (1 per box)

    tf_out = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_out



In [6]:
save_dir='/hdd3/dhj_container/DB/SSD_data_TFR/'
if not tf.gfile.Exists(save_dir):
    tf.gfile.MakeDirs(save_dir)
    
    
writer = tf.python_io.TFRecordWriter(save_dir+'train_data.tfrecord')

# TODO(user): Write code to read in your dataset to examples variable

for i in range(len(train)):
    tf_example = create_tf(train[i])
    writer.write(tf_example.SerializeToString())
writer.close()


writer = tf.python_io.TFRecordWriter(save_dir+'test_data.tfrecord')
for i in range(len(test)):
    tf_example = create_tf(test[i])
    writer.write(tf_example.SerializeToString())
writer.close()